In [1]:
import sqlite3

In [2]:
connection= sqlite3.connect("mydb.db")

In [3]:
connection

In [10]:
table_creation_query="""
CREATE TABLE IF NOT EXISTS employees(
    Emp_id INTEGER PRIMARY KEY,
    First_Name TEXT NOT NULL,
    Last_Name TEXT NOT NULL,
    Email TEXT UNIQUE NOT NULL,
    Hire_Date TEXT NOT NULL,
    Salary REAL NOT NULL
    );
    """

In [11]:
table_creation_query2 = """
CREATE TABLE IF NOT EXISTS customers(
    Customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    First_Name TEXT NOT NULL,
    Last_Name  TEXT NOT NULL,
    Email TEXT UNIQUE NOT NULL,
    Phone TEXT
    );
    """

In [15]:
table_creation_query3 = """
CREATE TABLE IF NOT EXISTS orders(
Order_id INTEGER PRIMARY KEY AUTOINCREMENT,
Customer_id INTEGER NOT NULL,
Order_Date TEXT NOT NULL,
Amount REAL NOT NULL,
FOREIGN KEY (Customer_id) REFERENCES customers(Customer_id));
"""

In [16]:
cursor = connection.cursor()

In [17]:
cursor.execute(table_creation_query)
cursor.execute(table_creation_query2)
cursor.execute(table_creation_query3)

In [21]:
insert_query_employees = """
INSERT INTO employees(Emp_id,First_Name,Last_Name,Email,Hire_Date,Salary)
VALUES(?,?,?,?,?,?)
"""
insert_query_customers = """
INSERT INTO customers(Customer_id,First_Name,Last_Name,Email,Phone)
VALUES(?,?,?,?,?)
"""

insert_query_orders = """
INSERT INTO orders(Order_id,Customer_id,Order_Date,Amount)
VALUES(?,?,?,?)
"""

In [33]:
employee_data = [
    (1,"ANANDKUMAR","SIVALANKA","anandkumars@gmail.com","12-06-2022",50000.00),
    (2,"Priyanka","SIVALANKA","priyanka@gmail.com","10-08-2023",40000.00),
    (3,"Aaron","lexi","lexi@gmail.com","09-10-2023",60000.00),
    (4,"Lexgantion","mongo","mongo@gmail.com","04-12-2023",80000.00),
]

customers_data = [
    (1,"Andy","Doe","andy@gmail.com","1234567892"),
    (2,"thron","pinky","thron@gmail.com","3456789233"),
    (3,"yummy","oil","oil@gmail.com","4567834222"),
    (4,"egg","milky","milky@gmail.com","3456782342"),
]

orders_data = [
    (1,1,"2023-12-01",250.25),
    (2,2,"2023-11-03",150.25),
    (3,3,"2023-10-05",300.25),
    (4,4,"2023-09-09",450.25),
]

In [39]:
insert_query = """
INSERT INTO employees (Emp_id, First_Name, Last_Name, Email, Hire_Date, Salary)
VALUES (?, ?, ?, ?, ?, ?)
ON CONFLICT(Emp_id) DO UPDATE SET
    First_Name = excluded.First_Name,
    Last_Name = excluded.Last_Name,
    Email = excluded.Email,
    Hire_Date = excluded.Hire_Date,
    Salary = excluded.Salary
"""
cursor.executemany(insert_query, employee_data)




In [41]:
insert_query = """
INSERT INTO customers (Customer_id,First_Name,Last_Name,Email,Phone)
VALUES (?, ?, ?, ?, ?)
ON CONFLICT(Customer_id) DO UPDATE SET
    First_Name = excluded.First_Name,
    Last_Name = excluded.Last_Name,
    Email = excluded.Email,
    Phone = excluded.Phone
    
"""
cursor.executemany(insert_query,customers_data )



In [42]:
insert_query = """
INSERT INTO orders(Order_id,Customer_id,Order_Date,Amount)
VALUES (?, ?, ?, ?)
ON CONFLICT(Order_id) DO UPDATE SET
   Customer_id= excluded.Customer_id,
   Order_Date= excluded.Order_Date,
   Amount = excluded.Amount
   
"""
cursor.executemany(insert_query, orders_data)

In [44]:
connection.commit()

In [45]:
cursor.execute("SELECT * from employees")

In [46]:
for row in cursor.fetchall():
    print(row)

(1, 'ANANDKUMAR', 'SIVALANKA', 'anandkumars@gmail.com', '12-06-2022', 50000.0)
(2, 'Priyanka', 'SIVALANKA', 'priyanka@gmail.com', '10-08-2023', 40000.0)
(3, 'Aaron', 'lexi', 'lexi@gmail.com', '09-10-2023', 60000.0)
(4, 'Lexgantion', 'mongo', 'mongo@gmail.com', '04-12-2023', 80000.0)


In [47]:
from langchain_community.utilities import SQLDatabase

In [48]:
db=SQLDatabase.from_uri("sqlite:///mydb.db")

In [49]:
db

In [50]:
db.dialect

'sqlite'

In [51]:
db.get_usable_table_names()

['customers', 'employees', 'orders']

In [53]:
import os 
from dotenv import load_dotenv


In [54]:
load_dotenv()
api_key = os.getenv("Groq_Api_key")

if not api_key:
    raise ValueError("Groq_Api_key is not set in the environment")

In [55]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192",api_key=api_key)

In [56]:
llm.invoke("how are you")

AIMessage(content="I'm just a language model, I don't have feelings or emotions like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 13, 'total_tokens': 59, 'completion_time': 0.131428571, 'prompt_time': 0.004331953, 'queue_time': 0.018358316, 'total_time': 0.135760524}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_7ab5f7e105', 'finish_reason': 'stop', 'logprobs': None}, id='run-ceb22ce7-4d66-4a6f-a4ad-3dc98b709d9e-0', usage_metadata={'input_tokens': 13, 'output_tokens': 46, 'total_tokens': 59})

In [72]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [74]:
toolkit =SQLDatabaseToolkit(db=db,llm=llm)

In [75]:
toolkit

SQLDatabaseToolkit(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1090b0a40>, llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10a5c2d80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10a6165d0>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=SecretStr('**********')))

In [77]:
tools = toolkit.get_tools()
for tool in tools:
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [78]:
list_tables_tool=next((tool for tool in tools if tool.name == "sql_db_list_tables"),None)

In [79]:
list_tables_tool

ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1090b0a40>)

In [83]:
get_schema_tool=next((tool for tool in tools if tool.name == "sql_db_schema"),None)

In [84]:
get_schema_tool

InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1090b0a40>)

In [85]:
get_query_checker_tool=next((tool for tool in tools if tool.name == "sql_db_query_checker"),None)

In [86]:
get_query_checker_tool

QuerySQLCheckerTool(description='Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1090b0a40>, llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10a5c2d80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10a6165d0>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=SecretStr('**********')), llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['dialect', 'query'], input_types={}, partial_variables={}, template='\n{query}\nDouble check the {dialect} query above for common mistakes, including:\n- Using NOT IN with NULL values\n- Using UNION when UNION ALL should have been used\n- Using BETWEEN for exclusive ranges\n- Data type mismatch in predicates\n- Properly quoting identifiers\n- Using the correct number of arguments for functions\n- Castin

In [88]:
list_tables_tool.invoke("")

'customers, employees, orders'

In [90]:
print(get_schema_tool.invoke("customers"))


CREATE TABLE customers (
	"Customer_id" INTEGER, 
	"First_Name" TEXT NOT NULL, 
	"Last_Name" TEXT NOT NULL, 
	"Email" TEXT NOT NULL, 
	"Phone" TEXT, 
	PRIMARY KEY ("Customer_id"), 
	UNIQUE ("Email")
)

/*
3 rows from customers table:
Customer_id	First_Name	Last_Name	Email	Phone
1	Andy	Doe	andy@gmail.com	1234567892
2	thron	pinky	thron@gmail.com	3456789233
3	yummy	oil	oil@gmail.com	4567834222
*/


In [91]:
from langchain_core.tools import tool 
@tool
def db_query_tool(query:str)->str:
    """ 
    Execute a SQL query against the database abd return the result.
    if the query is invalid or returns no results, an error message will be returned
    In case of an error,the user is advised to rewrite the query and try again.
    """
    result = db.run_no_throw(query)
    if not result:
        return "Error:Query failed.Please rewrite your query and try again"
    return result
    

In [92]:
db_query_tool.invoke("SELECT * FROM employees")

"[(1, 'ANANDKUMAR', 'SIVALANKA', 'anandkumars@gmail.com', '12-06-2022', 50000.0), (2, 'Priyanka', 'SIVALANKA', 'priyanka@gmail.com', '10-08-2023', 40000.0), (3, 'Aaron', 'lexi', 'lexi@gmail.com', '09-10-2023', 60000.0), (4, 'Lexgantion', 'mongo', 'mongo@gmail.com', '04-12-2023', 80000.0)]"

In [93]:
db.run("SELECT * FROM employees")

"[(1, 'ANANDKUMAR', 'SIVALANKA', 'anandkumars@gmail.com', '12-06-2022', 50000.0), (2, 'Priyanka', 'SIVALANKA', 'priyanka@gmail.com', '10-08-2023', 40000.0), (3, 'Aaron', 'lexi', 'lexi@gmail.com', '09-10-2023', 60000.0), (4, 'Lexgantion', 'mongo', 'mongo@gmail.com', '04-12-2023', 80000.0)]"